In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
from os import path
import shutil
import tabula
import glob
import pikepdf
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import io
import re

## Script to find the newst file in the folder

In [8]:
src = "C:\\Users\\Admin\\Downloads"
dst = "D:\\Gem1"
def latest_download_file():
      path = src
      os.chdir(path)
      files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
      newest = files[-1]

      return newest

## Script to download the file from GEM website and rename it to tender number

In [4]:
ser=Service(executable_path='C:\\edgedriver_win64\\msedgedriver.exe')
driver=webdriver.Edge(service=ser)
driver.maximize_window()
driver.get("https://bidplus.gem.gov.in/bidlists?bidlists")
file=''
i=1
while i<2:
    if i==1:
        driver.find_element(By.XPATH,'//*[@id="pagination"]/ul/li/ul/li[2]/a').click()
        

        for j in range(1,10):
            # Extracting the text from the link element
            link=driver.find_element(By.XPATH,f'//*[@id="pagi_content"]/div[{j}]/div[1]/p[1]/a').text
            link=link.replace("/","")
            link=f'{link}.pdf'
            # click on the link and download the file to destination.
            driver.find_element(By.XPATH,f'//*[@id="pagi_content"]/div[{j}]/div[1]/p[1]/a').click()
            
            time.sleep(3) 
            #identifying the latest downloaded file in source directory
            file=latest_download_file()
            # renaming the file with tender no which got from link text
            if file.startswith("GeM") and path.isfile(path.join(src, file)):
                os.rename(file,link)
        i=i+1
                
    else:
        time.sleep(3)
        driver.find_element(By.XPATH,'//*[@id="pagination"]/ul/li/ul/li[5]/a').click()
            
        for x in range(1,10):
                
                time.sleep(3)
                link=driver.find_element(By.XPATH,f'//*[@id="pagi_content"]/div[{x}]/div[1]/p[1]/a').text
                link=link.replace("/","")
                link=f'{link}.pdf'
                driver.find_element(By.XPATH,f'//*[@id="pagi_content"]/div[{x}]/div[1]/p[1]/a').click()
                time.sleep(3) 

                file=latest_download_file()
                try:
                    if file.startswith("GeM") and path.isfile(path.join(src, file)):
                        os.rename(file,link) 
                except:
                    pass
        i=i+1

### Script to extract text using PDF miner

In [5]:
def extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            
            text = fake_file_handle.getvalue()
            yield text
    
            # close open handles
            converter.close()
            fake_file_handle.close()
    
def extract_text(pdf_path):
    tot = {}
    for i, page in enumerate(extract_text_by_page(pdf_path)):
        tot.update({i+1 : page})
    return tot

In [6]:
files = [i for i in os.listdir(src) if i.startswith("GEM") and path.isfile(path.join(src, i))]
for f in files:
    shutil.move(path.join(src, f), dst)

In [22]:
src='D:/Gem1/'

In [9]:
import re

In [28]:
pdf_file_list=glob.glob(r'D:/Gem1/*.pdf')

In [29]:
pdf_file_list

['D:/Gem1\\GEM2022B1994279.pdf', 'D:/Gem1\\GEM2022B2005585.pdf']

In [12]:
for a in pdf_file_list:
    
    tender_file_name=a.split('\\')[1].replace('.pdf','')
    df = tabula.read_pdf(a, pages="1", stream=True)
    pdftext=extract_text(a)
    pattern='Dated: (\d{2}-\d{2}-\d{4})'
    tender_pattern='Bid Number:(\s\S*\d)'
    tender_date=re.findall(pattern, pdftext[1])
    tender_date=tender_date[0]
    tender_no=re.findall(tender_pattern, pdftext[1])[0].strip()
    
    
    
    try:
        for i in range (1,len(pdftext)+1):
            pattern3='DocumentEMD DetailRequired([a-zA-Z]*)'
            Emd=re.findall(pattern3,pdftext[i])
            if (len(Emd))>0:
                Emd=Emd[0]
                break
    except:
            Emd="NA"
        
        
    try:
        for i in range (1,len(pdftext)+1):
            pattern2='ePBG DetailRequired([a-zA-Z]*)'
            ePBG=re.findall(pattern2,pdftext[i])
            if (len(ePBG))>0:
                ePBG=ePBG[0]
                break
    except:
            ePBG="NA"
            
    try:    
        for i in range (1,len(pdftext)+1):    
            pattern1='Delivery Days1\**([\w][\D]*)'
            place=re.findall(pattern1,pdftext[i])
            if (len(place))>0:
                place=place[0]
                break
    except:
            place="NA"
        
    
    try:
        mseindex=df[0][df[0]['Unnamed: 0'].str.contains('MSE', regex=False, case=False, na=False)].index.tolist()
        msevalue=df[0]['Bid Details'][mseindex[0]+1]
    except:
        msevalue='No'
        
    new_df=[[            
            tende_no,tender_file_name,tender_date,place,ePBG,Emd,df[0]['Bid Details'][0],
            df[0]['Bid Details'][1],df[0]['Bid Details'][2],df[0]['Bid Details'][3],
         df[0]['Bid Details'][4],df[0]['Bid Details'][5],df[0]['Bid Details'][6],
        
         df[0]['Bid Details'][8],df[0]['Bid Details'][9],
         msevalue
  
       ]]
    columns=['tender_no','tender_file_name','Date','place_of_supply','ePBG','Emd','Bid End Date/Time','Bid Opening Date/Time','Bid Life Cycle (From Publish Date)','Bid Offer Validity (From End Date)',
        'Department Name','Organisation Name','office Name','Total Quantity','Item Category','MSE Exemption']
    new_df=pd.DataFrame(new_df,columns=columns)
    
    if os.path.isfile("D:\\Gem1\\gem1.csv"):        
            new_df.to_csv("D:\\Gem1\\gem1.csv",index=False,header=False,mode='a')            
    else:
             new_df.to_csv("D:\\Gem1\\gem1.csv",index=False)

## Downloading files from each tender

In [2]:

urls=[]
for file in pdf_file_list:
    print(file)
    pdf_file=pikepdf.Pdf.open(file)
    tender_no=file.split('\\')[1].replace('.pdf','')
    directory = tender_no
    parent_dir = r"D:\\Gem1"
    folder_location = os.path.join(parent_dir, directory)
    if not os.path.exists (folder_location):
        os.mkdir(folder_location)
    file.close()
    for page in pdf_file.pages:
        for annots in page.get("/Annots"):
            try:
                uri=annots.get("/A").get("/URI")
            except:
                pass
            if uri is not None:
                #print("[+] URL founs",uri)
                #print(str(uri))
                # do not download the file with terms and conditions
                if str(uri).endswith("termsCondition"):
                    pass
                else:
                    if str(uri).endswith(".pdf") or str(uri).endswith(".csv") or str(uri).endswith(".docx") or str(uri).endswith(".xlsx") :
                        filename=os.path.join(folder_location,str(uri).split("/")[-1])
                        r=requests.get(str(uri),stream=True)
                        print("Saving to", filename)
                        with open (filename,'wb') as f:
                            f.write(r.content)
                             
                    else:
                        try:
                            # Download the table directly from website
                            html=requests.get(str(uri))
                            soup=BeautifulSoup(html.text, 'lxml')
                            tables = soup.find_all('table')
                            df= pd.read_html(str(tables[0]))[0]
                            folder=f'D:\\Gem\\{tender_no}\\'
                            df.to_csv(f"{folder}\\{tender_no}.csv")
                        except:
                            pass
                        
              

NameError: name 'pdf_file_list' is not defined

### Moving Tender Files to folders

In [ ]:
pdf_file_list=glob.glob(r'D:/Gem1/*.pdf')
for file in pdf_file_list:
        source = file
        tender_no=file.split('\\')[1].replace('.pdf','')
  
        # Destination path
        destination = f"D:\\Gem1\\{tender_no}"
  
        # Move the content of
        # source to destination
        dest = shutil.move(source, destination)
  
        # print(dest) prints the 
        # Destination of moved directory